# NLP Analysis

### Top Words, Lemmatization, Topic Modeling

In [1]:
import praw
import pandas as pd
import re
import string
import emoji as e
import logging
import scattertext as st
import nltk
import spacy
import numpy as np
import scipy.sparse as ss

from sklearn import datasets
from sklearn.feature_extraction import text 
from sklearn.decomposition import TruncatedSVD,NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.tokenize import MWETokenizer, word_tokenize
from nltk import pos_tag
from gensim import corpora, models, matutils
from nltk.stem import WordNetLemmatizer, LancasterStemmer
from corextopic import corextopic as ct
from corextopic import vis_topic as vt
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
# Read in pickled files from notebook 2
autism_fem = pd.read_pickle('autism_fem.pkl')
autism_gen = pd.read_pickle('autism_gen.pkl')
autism_fem.shape,autism_gen.shape

((5778, 21), (7341, 21))

In [3]:
autism_fem.head(10)

,created_utc,title,selftext,full_text,spacy_doc,full_clean,nouns,adj,verb,propn,...,upvote_ratio,num_comments,ID,author,name,verb_lem,noun_lem,sentiment_score,full_text_nouns,category
0,2021-02-09 23:20:33,Origin Story Megathread,We get a lot of foot traffic from people who a...,Origin Story Megathread. We get a lot of foot ...,"(Origin, Story, Megathread, ., We, get, a, lot...",origin story megathread we get a lot of foot ...,lot foot traffic people process autism diagnos...,turbulent right medical individual new unusual...,get discovering investigating considering goin...,Origin Story Megathread,...,1.00,306,lggazi,idungonwent,t3_lggazi,get discover investigate consider go want know...,lot foot traffic people process autism diagnos...,0.9324,lot foot traffic process discovering investiga...,fem
1,2022-04-16 16:40:16,Study finds indicators that autistic people co...,**Lay abstract** \nSharing information with o...,Study finds indicators that autistic people co...,"(Study, finds, indicators, that, autistic, peo...",study finds indicators that autistic people co...,Study indicators people people communication d...,autistic fellow autistic allistic autistic abs...,finds commicate arise try communicate Sharing ...,Lay,...,0.98,26,u51o7g,Hoihe,t3_u51o7g,find commicate arise try communicate sharing r...,study indicator people people communication de...,0.9775,communication information ability communicatio...,fem
2,2022-04-16 17:38:28,"so, are your parents terrorizing you too?",lol i am trying to hold my tears back while wr...,"so, are your parents terrorizing you too?. lol...","(so, ,, are, your, parents, terrorizing, you, ...",so are your parents terrorizing you too lol...,parents tears x200B possibility diagnosis ther...,difficult dominant sure nicer neutral few rude...,terrorizing lol trying hold writing discussing...,,...,0.97,10,u52wf1,dykenergy,t3_u52wf1,terrorize lol try hold write discussing say di...,parent tear x200b possibility diagnosis therap...,-0.9584,lol i x200B possibility diagnosis therapist ma...,fem
3,2022-04-16 13:52:13,what are some of your social interactions that...,so obviously its no rare occasion for any of u...,what are some of your social interactions that...,"(what, are, some, of, your, social, interactio...",what are some of your social interactions that...,interactions occasion misunderstood NTs intera...,social wrong rare social own social social due...,gone get considering views tend blow think try...,faux pas H K K J. J K J J i J,...,0.98,38,u4y9ji,petpuppy,t3_u4y9ji,go get consider view tend blow think try find ...,interaction occasion misunderstood NTs interac...,-0.8480,occasion misunderstood faux pas proportion hum...,fem
4,2022-04-16 16:44:05,Human rights activist bullied for her autistic...,"I’ve never heard about this woman before, but ...",Human rights activist bullied for her autistic...,"(Human, rights, activist, bullied, for, her, a...",human rights activist bullied for her autistic...,rights activist traits woman People people thi...,Human autistic furious autistic weird childish...,bullied heard got read treated call try ’m hea...,Greta Thunberg 😡 https://hireup.com.au/news/gr...,...,0.93,2,u51r1d,Bitter-Statement6471,t3_u51r1d,bully hear got read treat call try ' m hear say ’,right activist trait woman People people thing...,-0.4019,activist woman sure https,fem
5,2022-04-16 15:30:13,How do I tell my partner I don’t want to be hu...,I recently told my partner about my stims and ...,How do I tell my partner I don’t want to be hu...,"(How, do, I, tell, my, partner, I, do, n’t, wa...",how do i tell my partner i don t want to be hu...,partner partner stims relationship work stims ...,able fine happy first due crazy first scared able,tell want hugged stim told said want do said w...,,...,1.00,5,u507ci,Sweaty-Ad-3526,t3_u507ci,tell want hug stim tell say want do say want h...,partner partner stims relationship work stims ...,0.9808,partner t partner relationship anxiety enjoyme...,fem
6,20

### Text Corpus

In [4]:
# Create several different corpuses of all the text
corpus_f = list(autism_fem.full_clean)
corpus_g = list(autism_gen.full_clean)

corpus_f_noun = list(autism_fem.nouns)
corpus_g_noun = list(autism_gen.nouns)

corpus_f_adj = list(autism_fem.adj)
corpus_g_adj = list(autism_gen.adj)

corpus_f_noun_adj = list(autism_fem.adj + autism_fem.nouns)
corpus_g_noun_adj = list(autism_gen.adj + autism_gen.nouns)

corpus_f_noun_verb = list(autism_fem.verb + autism_fem.nouns)
corpus_g_noun_verb = list(autism_gen.verb + autism_gen.nouns)

### Top words in dataset

Interpretation: There is a high similarity in the overall top words between the two datasets. There are also a lot of repeated words (like, people, just, etc.), which don't give much information about the datasets. I also didn't remove word fragments and common words yet (don, ve, autism, autistic, etc.)

In [5]:
# Create document term matrix from corpus
cv = CountVectorizer(stop_words='english',min_df=0.01)
X = cv.fit_transform(corpus_f)
dtm_f = pd.DataFrame(X.toarray(), columns=cv.get_feature_names())

# Create dtm with totals in entire corpus
total_dtm_f = dtm_f.sum()  
total_dtm_f.sort_values(ascending=False).head(20)

like        8947
just        8140
people      5715
don         5627
feel        5363
know        4418
ve          4098
really      3718
time        3548
autistic    3105
things      2828
autism      2820
want        2763
think       2698
life        2030
work        2009
way         1996
lot         1983
make        1882
didn        1642
dtype: int64

In [6]:
# Create document term matrix from corpus
cv = CountVectorizer(stop_words='english',min_df=0.01)
X = cv.fit_transform(corpus_g)
dtm_g = pd.DataFrame(X.toarray(), columns=cv.get_feature_names())

# Create dtm with totals in entire corpus
total_dtm_g = dtm_g.sum()  
total_dtm_g.sort_values(ascending=False).head(20)

like        8843
just        7900
people      6742
don         5642
feel        4595
know        4512
ve          3946
really      3482
autism      3397
time        3296
autistic    3210
want        2886
things      2858
think       2831
life        2142
make        1967
way         1955
does        1820
lot         1812
friends     1732
dtype: int64

### Top Adjectives

In [7]:
# Create document term matrix from corpus
cv = CountVectorizer(stop_words='english',min_df=0.01)
X = cv.fit_transform(corpus_f_adj)
dtm_f = pd.DataFrame(X.toarray(), columns=cv.get_feature_names())

# Create dtm with totals in entire corpus
total_dtm_f = dtm_f.sum()  
total_dtm_f.sort_values(ascending=False).head(10)

autistic     2956
social       1545
good         1315
able          951
new           936
bad           913
hard          907
sure          886
different     836
little        768
dtype: int64

In [8]:
# Create document term matrix from corpus
cv = CountVectorizer(stop_words='english',min_df=0.01)
X = cv.fit_transform(corpus_g_adj)
dtm_g = pd.DataFrame(X.toarray(), columns=cv.get_feature_names())

# Create dtm with totals in entire corpus
total_dtm_g = dtm_g.sum()  
total_dtm_g.sort_values(ascending=False).head(10)

autistic     3042
social       1570
good         1447
bad          1065
able          921
different     912
hard          872
sure          843
special       836
new           787
dtype: int64

### Top Nouns

In [9]:
# Create document term matrix from corpus
cv = CountVectorizer(stop_words='english',min_df=0.01)
X = cv.fit_transform(corpus_f_noun)
dtm_f = pd.DataFrame(X.toarray(), columns=cv.get_feature_names())

# Create dtm with totals in entire corpus
total_dtm_f = dtm_f.sum()  
total_dtm_f.sort_values(ascending=False).head(10)

people     5714
time       3527
things     2829
autism     2602
life       2027
lot        1944
way        1872
friends    1582
day        1421
person     1377
dtype: int64

In [10]:
# Create document term matrix from corpus
cv = CountVectorizer(stop_words='english',min_df=0.01)
X = cv.fit_transform(corpus_g_noun)
dtm_g = pd.DataFrame(X.toarray(), columns=cv.get_feature_names())

# Create dtm with totals in entire corpus
total_dtm_g = dtm_g.sum()  
total_dtm_g.sort_values(ascending=False).head(10)

people     6740
time       3278
autism     3079
things     2860
life       2135
way        1844
lot        1774
friends    1723
person     1544
years      1492
dtype: int64

### Nop Nouns + Adjectives

In [11]:
# Create document term matrix from corpus
cv = CountVectorizer(stop_words='english',min_df=0.01)
X = cv.fit_transform(corpus_f_noun_adj)
dtm_f = pd.DataFrame(X.toarray(), columns=cv.get_feature_names())

# Create dtm with totals in entire corpus
total_dtm_f = dtm_f.sum()  
total_dtm_f.sort_values(ascending=False).head(10)

people      5473
time        3437
autistic    2853
things      2768
autism      2415
life        1967
lot         1905
way         1843
friends     1515
social      1486
dtype: int64

In [12]:
# Create document term matrix from corpus
cv = CountVectorizer(stop_words='english',min_df=0.01)
X = cv.fit_transform(corpus_g_noun_adj)
dtm_g = pd.DataFrame(X.toarray(), columns=cv.get_feature_names())

# Create dtm with totals in entire corpus
total_dtm_g = dtm_g.sum()  
total_dtm_g.sort_values(ascending=False).head(10)

people      6379
time        3213
autistic    2889
autism      2849
things      2778
life        2068
way         1801
lot         1730
friends     1649
person      1493
dtype: int64

### Top Nouns + Verbs

In [13]:
# Create document term matrix from corpus
cv = CountVectorizer(stop_words='english',min_df=0.01)
X = cv.fit_transform(corpus_f_noun_verb)
dtm_f = pd.DataFrame(X.toarray(), columns=cv.get_feature_names())

# Create dtm with totals in entire corpus
total_dtm_f = dtm_f.sum()  
total_dtm_f.sort_values(ascending=False).head(20)

people     5477
feel       5161
know       4312
time       3439
things     2767
want       2725
think      2624
autism     2419
life       1968
work       1930
lot        1906
way        1845
make       1824
need       1574
friends    1513
going      1494
help       1446
say        1431
said       1406
day        1393
dtype: int64

In [14]:
# Create document term matrix from corpus
cv = CountVectorizer(stop_words='english',min_df=0.01)
X = cv.fit_transform(corpus_g_noun_verb)
dtm_g = pd.DataFrame(X.toarray(), columns=cv.get_feature_names())

# Create dtm with totals in entire corpus
total_dtm_g = dtm_g.sum()  
total_dtm_g.sort_values(ascending=False).head(10)

people    6374
feel      4388
know      4368
time      3213
autism    2847
want      2836
things    2777
think     2720
life      2065
make      1899
dtype: int64

# Lemmatization

In [15]:
# autism_fem['verb_lem'] = autism_fem.verb.copy()


In [16]:
nlp = spacy.load("en_core_web_md")

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

autism_fem['verb_lem'] = autism_fem["verb"].apply(lambda row: " ".join([w.lemma_ for w in nlp(row)]))
autism_gen['verb_lem'] = autism_gen["verb"].apply(lambda row: " ".join([w.lemma_ for w in nlp(row)]))


In [17]:
autism_fem['noun_lem'] = autism_fem["nouns"].apply(lambda row: " ".join([w.lemma_ for w in nlp(row)]))
autism_gen['noun_lem'] = autism_gen["nouns"].apply(lambda row: " ".join([w.lemma_ for w in nlp(row)]))


# Topic Modeling

I decided to use nouns + verbs for topic modeling because these are the parts of speech most likely to represent clear topics. I made sure the verbs were lemmatized to help improve topic modeling.

### Latent Semantic Analysis

In [18]:
corpus_f_noun_verb_lem = list(autism_fem.verb_lem + autism_fem.noun_lem)
corpus_g_noun_verb_lem = list(autism_gen.verb_lem + autism_gen.noun_lem)


In [19]:
# Create document term matrix again from corpus
vectorizer = CountVectorizer(stop_words='english',min_df=0.01)
doc_term = vectorizer.fit_transform(corpus_f_noun_verb_lem)
doc_term.shape

(5778, 687)

In [20]:
# Make dtm into a df
doc_term_df = pd.DataFrame(doc_term.toarray(), index=corpus_f_noun_verb_lem, columns=vectorizer.get_feature_names())

In [21]:
# Fit an LSA model using TruncatedSVD
lsa = TruncatedSVD(20)
lsa.fit(doc_term)
lsa

TruncatedSVD(n_components=20)

In [22]:
topic_term = lsa.components_.round(3)
topic_term

array([[ 0.008,  0.008,  0.015, ...,  0.099,  0.004,  0.007],
       [-0.   ,  0.001, -0.   , ..., -0.026,  0.003, -0.004],
       [-0.001,  0.006,  0.003, ...,  0.097,  0.007,  0.01 ],
       ...,
       [ 0.002, -0.014, -0.002, ..., -0.201, -0.013,  0.012],
       [ 0.022, -0.006, -0.014, ..., -0.198, -0.004,  0.008],
       [ 0.021, -0.002,  0.012, ..., -0.113,  0.   , -0.002]])

In [23]:
topic_term_df = pd.DataFrame(topic_term.round(3),
                index = ["component_1", "component_2", "component_3", "component_4", "component_5",
                        "component_6", "component_7", "component_8", "component_9", "component_10",
                        "component_11", "component_12", "component_13", "component_14", "component_15",
                        "component_16", "component_17", "component_18", "component_19", "component_20"], 
                columns = vectorizer.get_feature_names())
topic_term_df

,ability,abuse,accept,access,accommodation,account,acknowledge,act,action,activity,...,worker,working,world,worry,write,writing,x200b,year,yell,yesterday
component_1,0.008,0.008,0.015,0.004,0.006,0.004,0.003,0.020,0.008,0.006,...,0.005,0.006,0.028,0.013,0.023,0.003,0.017,0.099,0.004,0.007
component_2,-0.000,0.001,-0.000,-0.003,-0.001,0.002,0.001,0.010,-0.002,-0.002,...,0.004,-0.008,0.002,-0.011,-0.001,0.001,0.007,-0.026,0.003,-0.004
component_3,-0.001,0.006,0.003,0.004,0.006,0.002,0.004,-0.007,-0.002,0.006,...,0.003,0.008,-0.014,0.003,0.021,-0.000,0.007,0.097,0.007,0.010
component_4,-0.009,-0.000,-0.008,-0.007,-0.016,-0.001,0.002,0.008,0.003,-0.012,...,-0.002,-0.017,-0.018,-0.007,0.001,-0.004,-0.019,-0.075,0.003,-0.002
component_5,0.012,0.009,0.019,0.008,0.002,0.003,0.001,0.002,0.003,0.012,...,-0.006,-0.003,0.007,0.012,0.012,0.003,-0.012,0.032,-0.003,-0.005
component_6,0.002,0.015,-0.002,-0.000,-0.005,0.003,-0.003,0.002,-0.001,0.001,...,-0.012,-0.008,0.024,0.002,-0.013,0.001,-0.001,0.078,-0.003,-0.013
component_7,-0.002,-0.002,0.001,-0.002,0.003,-0.002,0.002,0.012,0.013,0.003,...,-0.004,-0.007,-0.013,0.026,-0.000,0.002,0.003,-0.075,0.002,-0.002
component_8,0.002,0.001,0.000,0.002,-0.007,0.001,0.002,0.010,-0.002,0.009,...,-0.002,-0.004,-0.006,-0.014,-0.008,-0.001,0.002,0.071,0.000,-0.001
component_9,0.009,0.007,0.000,-0.002,-0.013,-0.001,0.003,0.003,0.003,-0.003,...,-0.007,-0.002,0.003,0.014,-0.013,-0.002,0.005,-0.029,-0.001,0.003
component_10,0.005,0.008,0.009,0.003,0.006,0.001,0.002,-0.011,0.006,0.009,...,-0.001,-0.003,0.005,-0.002,-0.009,0.006,-0.009,-0.043,0.001,0.001


In [24]:
# Function to display the top n terms in each topic
def display_topics(model, feature_names, no_top_words, topic_names = None): 
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix + 1)
        else:
            print("\nTopic: ", topic_names[ix])
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
    print("\n")
    return model, feature_names, no_top_words

In [25]:
output = display_topics(lsa, vectorizer.get_feature_names(), 10)



Topic  1
feel, people, know, make, thing, want, time, say, think, try

Topic  2
people, say, autism, think, talk, friend, tell, person, like, lot

Topic  3
say, time, want, work, know, tell, need, day, ask, help

Topic  4
say, know, tell, feel, thing, talk, ask, friend, want, understand

Topic  5
autism, diagnosis, diagnose, disorder, woman, spectrum, think, experience, self, behavior

Topic  6
know, want, friend, life, husband, relationship, thing, love, year, sex

Topic  7
thing, make, time, way, try, day, change, food, eat, understand

Topic  8
friend, time, talk, school, tell, autism, group, friendship, kid, year

Topic  9
time, say, husband, want, people, life, love, relationship, woman, feel

Topic  10
want, autism, make, work, love, relationship, husband, understand, disorder, try

Topic  11
make, try, need, husband, say, way, people, sense, mask, come

Topic  12
tell, try, need, help, life, talk, think, understand, way, live

Topic  13
think, friend, life, work, job, year, say

#### There is A LOT of overlap in words of the different topics. Let's try adding the top 20 most common words to the stop words list.

In [26]:
add_stop_words = ['like','feel','people','know','make','want','say','time','think','try','things','work','tell','autism',
                  'talk','life','need','way','lot','help','ask', 'autistic', 'asd', 'don','t', 'i','m', 've', 'they',
                  're', 'd', 'isn', 'll', 'what','s', 'that', 'he', 'she','a', 'didn', 'wasn','x200b']

# Add new stop words
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)


In [27]:
# Create document term matrix again from corpus
vectorizer = CountVectorizer(stop_words=stop_words,min_df=0.01)
doc_term = vectorizer.fit_transform(corpus_f_noun_verb_lem)
doc_term.shape

# Make dtm into a df
doc_term_df = pd.DataFrame(doc_term.toarray(), index=corpus_f_noun_verb_lem, columns=vectorizer.get_feature_names())

# Fit an LSA model using TruncatedSVD
lsa = TruncatedSVD(20)
lsa.fit(doc_term)

topic_term = lsa.components_.round(3)

topic_term_df = pd.DataFrame(topic_term.round(3),
                index = ["component_1", "component_2", "component_3", "component_4", "component_5",
                        "component_6", "component_7", "component_8", "component_9", "component_10",
                        "component_11", "component_12", "component_13", "component_14", "component_15",
                        "component_16", "component_17", "component_18", "component_19", "component_20"], 
                columns = vectorizer.get_feature_names())


In [28]:
# Function to display the top n terms in each topic
def display_topics(model, feature_names, no_top_words, topic_names = None): 
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix + 1)
        else:
            print("\nTopic: ", topic_names[ix])
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
    print("\n")
    return model, feature_names, no_top_words

In [29]:
output = display_topics(lsa, vectorizer.get_feature_names(), 10)



Topic  1
thing, friend, year, day, understand, start, come, person, look, love

Topic  2
thing, day, job, question, brain, diagnosis, eat, answer, experience, food

Topic  3
diagnosis, job, year, diagnose, day, experience, disorder, woman, self, spectrum

Topic  4
diagnosis, understand, diagnose, woman, disorder, experience, behavior, self, spectrum, relationship

Topic  5
husband, love, relationship, person, sex, day, come, end, understand, partner

Topic  6
day, use, food, eat, kid, school, wear, room, child, mom

Topic  7
school, year, husband, kid, love, parent, relationship, mom, look, family

Topic  8
job, disorder, use, behavior, school, communication, language, spectrum, child, interaction

Topic  9
diagnosis, disorder, day, thing, anxiety, spectrum, friend, husband, behavior, diagnose

Topic  10
anxiety, school, live, feeling, learn, answer, cause, self, issue, thought

Topic  11
start, eye, look, person, eat, stop, appointment, year, understand, family

Topic  12
look, eye, 

#### The topics are still not clear, but adding common words as stop words does help differentiate them more.

### Non-Negative Matrix Factorization (NMF)

In [30]:
# Create document term matrix again from corpus
vectorizer = CountVectorizer(stop_words=stop_words,min_df=0.01 ) # min_df=0.001 
doc_term = vectorizer.fit_transform(corpus_f_noun_verb_lem)
doc_term.shape

(5778, 663)

In [31]:
# Make dtm into a df
doc_term_df = pd.DataFrame(doc_term.toarray(), index=corpus_f_noun_verb_lem, columns=vectorizer.get_feature_names())
doc_term_df

,ability,abuse,accept,access,accommodation,account,acknowledge,act,action,activity,...,word,worker,working,world,worry,write,writing,year,yell,yesterday
get discover investigate consider go want know give do go help share see resonate tell Regale discover see learn present realize fit make realize golot foot traffic people process autism diagnosis time people discovery process track professional advice people process experience people story comment origin story tale world superpower Questions lot way autism girl spectrum diagnosis diagnosis process,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
find commicate arise try communicate sharing relie communicate define expected find share want find sharing recruit tell ask share share hear look shared find share do be share ask feel get experienced sharing show have share experience be interactstudy indicator people people communication deficit people information people ability Autism deficit communication people information people case people information people people group people group group group groups group group person group story person person group story detail story stage people information people people people group people information Participants person interaction people group rapport person story find people skill information rapport problem people https://journals.sagepub.com/doi/full/10.1177/1362361320919286,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
terrorize lol try hold write discussing say diagnosed think learn master grow demanding mean start learn unmask tell stop move grab stop move tell stop rock stop move tell ask answered TALKING try explain reflect think agree tell stare touch hug state work crossing seem understand feel looseparent tear x200b possibility diagnosis therapist mask parent obedience hand hand face tone minute mum question tone feeling problem father consent therapy problem boundary presence mind,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
go get consider view tend blow think try find fail repeat have have sit talk mind sit mind m eavesdrop hear eavesdropping mention pass keep trigger talking sit mention have remember hear guess have do mean try make noticed got accuse finding do explain confused look mention explain remember think m accuse take do reflect discuss guess add m relive laugh wallowinteraction occasion misunderstood NTs interaction interaction rule proportion misunderstand humor example school science grade year friend girl acquaintance friend class friend observer participator friend troublemaker guy class i d month incident mother choice un - time corner mother cancer month look face smirk face time sense piece information cancer info eavesdrop part conversation month month addition eavesdropping memory morning cancer body language topic story cringe morning cringe lol,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
bully hear got read treat call try ' m hear say ’right activist trait woman People people thing People scumbag,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
love watchmusic philosophy,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
notice change tie use wear make overloaded feel have isolate find mask hide ' m ’s make feel know ’ m recall cause do recalibrateoverload age thing hair bra glass control pandemic clothing tad bit time kid age thing thing overload change time,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
wonder have talking learn having haveeye contact eye contact mask eye contact child,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
find believe going appreciate drop get deal work give have try find go have prove know find suppose do seem create runne make find get sell have validate see pursue end do try adapt design dealpurpose life kind life crisis advice college time College help way college job paper stuff qualification experience purpose life world world neurotypicals sense guy purpose type job art number science degree knowledge t

In [32]:
nmf = NMF(20, init = "nndsvda")
nmf.fit(doc_term_df)

NMF(init='nndsvda', n_components=20)

In [33]:
topic_term = nmf.components_.round(3)
topic_term.shape

(20, 663)

In [34]:
topic_term_df = pd.DataFrame(topic_term.round(3),
                index = ["component_1", "component_2", "component_3", "component_4", "component_5",
                        "component_6", "component_7", "component_8", "component_9", "component_10",
                        "component_11", "component_12", "component_13", "component_14", "component_15",
                        "component_16", "component_17", "component_18", "component_19", "component_20"],
                columns = vectorizer.get_feature_names())
topic_term_df

,ability,abuse,accept,access,accommodation,account,acknowledge,act,action,activity,...,word,worker,working,world,worry,write,writing,year,yell,yesterday
component_1,0.000,0.133,0.705,0.173,0.397,0.075,0.000,0.000,0.000,0.000,...,0.024,0.002,0.075,0.000,0.000,0.344,0.000,0.000,0.000,0.000
component_2,0.000,0.000,0.013,0.000,0.000,0.014,0.000,0.129,0.000,0.000,...,0.000,0.000,0.000,0.102,0.000,0.000,0.000,0.000,0.000,0.000
component_3,0.000,0.000,0.013,0.000,0.054,0.000,0.007,0.040,0.000,0.000,...,0.000,0.003,0.000,0.000,0.000,0.074,0.000,0.000,0.007,0.000
component_4,0.000,0.000,0.000,0.001,0.161,0.000,0.000,0.000,0.000,0.110,...,0.000,0.000,0.046,0.000,0.000,0.000,0.018,0.000,0.000,0.162
component_5,0.087,0.391,0.000,0.000,0.000,0.000,0.000,0.101,0.069,0.052,...,0.001,0.000,0.000,0.000,0.066,0.000,0.000,0.139,0.000,0.000
component_6,0.078,0.000,0.002,0.006,0.064,0.040,0.000,0.000,0.000,0.013,...,0.000,0.152,0.288,0.114,0.000,0.094,0.011,0.000,0.009,0.047
component_7,0.069,0.000,0.000,0.103,0.029,0.000,0.000,0.352,0.000,0.012,...,0.000,0.055,0.000,0.222,0.000,0.584,0.049,0.101,0.023,0.000
component_8,0.477,0.013,0.072,0.122,0.000,0.121,0.000,0.000,0.000,0.686,...,0.288,0.003,0.000,0.000,0.000,0.000,0.128,0.000,0.000,0.000
component_9,0.086,0.000,0.233,0.000,0.000,0.000,0.000,0.105,0.000,0.000,...,0.127,0.051,0.000,0.303,0.000,0.000,0.039,0.000,0.001,0.058
component_10,0.108,0.057,0.308,0.097,0.000,0.000,0.010,0.543,0.233,0.000,...,0.049,0.000,0.021,0.299,1.049,0.000,0.005,0.000,0.000,0.072


In [35]:
# Function to display the top n terms in each topic
def display_topics(model, feature_names, no_top_words, topic_names = None): 
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix + 1)
        else:
            print("\nTopic: ", topic_names[ix])
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
    print("\n")
    return model, feature_names, no_top_words

In [36]:
output = display_topics(nmf, vectorizer.get_feature_names(), 10)


Topic  1
diagnosis, diagnose, self, doctor, therapist, spectrum, adhd, test, assessment, research

Topic  2
friend, friendship, group, hang, party, relationship, meet, guy, care, family

Topic  3
thing, change, stuff, relate, happen, realize, explain, problem, thought, example

Topic  4
day, hour, week, wear, night, sleep, today, leave, house, plan

Topic  5
husband, relationship, sex, mask, end, come, change, issue, problem, point

Topic  6
job, interview, week, pay, boss, career, manager, apply, burn, end

Topic  7
school, class, teacher, girl, grade, child, read, kid, college, struggle

Topic  8
disorder, behavior, spectrum, communication, language, interaction, pattern, difficulty, share, example

Topic  9
understand, mean, explain, struggle, joke, conversation, sound, read, believe, learn

Topic  10
anxiety, feeling, live, experience, answer, learn, question, cause, thought, self

Topic  11
woman, experience, share, question, group, read, post, meeting, comment, gender

Topic  12

#### NMF seems to separate the topics much better than LSA

### Latent Dirichlet Allocation

In [37]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [38]:
vectorizer = CountVectorizer(stop_words=stop_words,min_df=0.01 ) # min_df=0.001 
doc_term = vectorizer.fit_transform(corpus_f_noun_verb_lem)

term_doc = doc_term.transpose()
corpus = matutils.Sparse2Corpus(term_doc)
id2word = dict((v, k) for k, v in vectorizer.vocabulary_.items())
lda = models.LdaModel(corpus=corpus, num_topics=15, id2word=id2word, passes=10)
lda

2022-05-17 09:03:39,995 : INFO : using symmetric alpha at 0.06666666666666667
2022-05-17 09:03:39,995 : INFO : using symmetric eta at 0.06666666666666667
2022-05-17 09:03:39,996 : INFO : using serial LDA version on this node
2022-05-17 09:03:39,997 : INFO : running online (multi-pass) LDA training, 15 topics, 10 passes over the supplied corpus of 5778 documents, updating model once every 2000 documents, evaluating perplexity every 5778 documents, iterating 50x with a convergence threshold of 0.001000
2022-05-17 09:03:40,011 : INFO : PROGRESS: pass 0, at document #2000/5778
2022-05-17 09:03:41,361 : INFO : merging changes from 2000 documents into a model of 5778 documents
2022-05-17 09:03:41,363 : INFO : topic #11 (0.067): 0.031*"friend" + 0.022*"thing" + 0.014*"person" + 0.011*"understand" + 0.011*"job" + 0.010*"post" + 0.010*"group" + 0.009*"share" + 0.009*"experience" + 0.009*"question"
2022-05-17 09:03:41,363 : INFO : topic #5 (0.067): 0.018*"thing" + 0.017*"week" + 0.017*"day" + 0.

2022-05-17 09:03:48,267 : INFO : topic #2 (0.067): 0.030*"thing" + 0.021*"sleep" + 0.021*"music" + 0.021*"day" + 0.016*"night" + 0.014*"play" + 0.014*"watch" + 0.013*"brain" + 0.012*"character" + 0.010*"love"
2022-05-17 09:03:48,267 : INFO : topic diff=0.393202, rho=0.452262
2022-05-17 09:03:48,279 : INFO : PROGRESS: pass 2, at document #2000/5778
2022-05-17 09:03:49,006 : INFO : merging changes from 2000 documents into a model of 5778 documents
2022-05-17 09:03:49,008 : INFO : topic #14 (0.067): 0.026*"school" + 0.024*"issue" + 0.021*"therapist" + 0.020*"experience" + 0.019*"therapy" + 0.018*"thing" + 0.016*"trait" + 0.014*"bully" + 0.014*"use" + 0.013*"look"
2022-05-17 09:03:49,009 : INFO : topic #10 (0.067): 0.042*"look" + 0.039*"thing" + 0.035*"wear" + 0.035*"eye" + 0.023*"contact" + 0.018*"face" + 0.013*"day" + 0.013*"sit" + 0.012*"hair" + 0.010*"issue"
2022-05-17 09:03:49,009 : INFO : topic #3 (0.067): 0.028*"friend" + 0.025*"year" + 0.020*"dog" + 0.018*"age" + 0.018*"use" + 0.01

2022-05-17 09:03:54,019 : INFO : topic diff=0.397200, rho=0.380997
2022-05-17 09:03:54,031 : INFO : PROGRESS: pass 4, at document #2000/5778
2022-05-17 09:03:54,659 : INFO : merging changes from 2000 documents into a model of 5778 documents
2022-05-17 09:03:54,660 : INFO : topic #3 (0.067): 0.031*"friend" + 0.029*"year" + 0.024*"age" + 0.023*"dog" + 0.019*"thing" + 0.017*"use" + 0.017*"thought" + 0.017*"adult" + 0.013*"walk" + 0.013*"come"
2022-05-17 09:03:54,660 : INFO : topic #12 (0.067): 0.072*"thing" + 0.019*"stop" + 0.017*"hand" + 0.016*"problem" + 0.015*"start" + 0.015*"school" + 0.014*"voice" + 0.014*"experience" + 0.013*"notice" + 0.012*"year"
2022-05-17 09:03:54,661 : INFO : topic #7 (0.067): 0.035*"read" + 0.026*"write" + 0.024*"school" + 0.019*"love" + 0.019*"book" + 0.019*"experience" + 0.016*"study" + 0.016*"class" + 0.015*"struggle" + 0.015*"post"
2022-05-17 09:03:54,662 : INFO : topic #10 (0.067): 0.052*"look" + 0.046*"eye" + 0.045*"wear" + 0.031*"thing" + 0.030*"contact

2022-05-17 09:04:00,079 : INFO : topic #14 (0.067): 0.034*"therapist" + 0.032*"therapy" + 0.031*"issue" + 0.027*"school" + 0.023*"trait" + 0.021*"experience" + 0.018*"bully" + 0.017*"thing" + 0.016*"remember" + 0.014*"empathy"
2022-05-17 09:04:00,079 : INFO : topic #4 (0.067): 0.030*"friend" + 0.027*"relationship" + 0.025*"thing" + 0.021*"love" + 0.021*"come" + 0.020*"feeling" + 0.020*"husband" + 0.019*"date" + 0.018*"person" + 0.016*"boyfriend"
2022-05-17 09:04:00,080 : INFO : topic #12 (0.067): 0.076*"thing" + 0.021*"hand" + 0.020*"stop" + 0.018*"school" + 0.017*"start" + 0.016*"problem" + 0.016*"notice" + 0.015*"experience" + 0.014*"voice" + 0.013*"stim"
2022-05-17 09:04:00,081 : INFO : topic #6 (0.067): 0.078*"eat" + 0.069*"food" + 0.036*"thing" + 0.023*"plan" + 0.021*"buy" + 0.020*"smell" + 0.019*"store" + 0.017*"clothe" + 0.016*"day" + 0.014*"texture"
2022-05-17 09:04:00,082 : INFO : topic #9 (0.067): 0.038*"mask" + 0.032*"relationship" + 0.031*"person" + 0.025*"thing" + 0.025*"l

2022-05-17 09:04:05,327 : INFO : topic #4 (0.067): 0.033*"friend" + 0.029*"relationship" + 0.025*"thing" + 0.023*"love" + 0.022*"feeling" + 0.022*"husband" + 0.022*"date" + 0.021*"come" + 0.019*"person" + 0.017*"boyfriend"
2022-05-17 09:04:05,328 : INFO : topic #12 (0.067): 0.078*"thing" + 0.023*"hand" + 0.020*"stop" + 0.020*"school" + 0.019*"start" + 0.018*"notice" + 0.016*"problem" + 0.016*"experience" + 0.015*"stim" + 0.013*"voice"
2022-05-17 09:04:05,328 : INFO : topic #10 (0.067): 0.058*"eye" + 0.057*"look" + 0.055*"wear" + 0.037*"hair" + 0.037*"contact" + 0.029*"face" + 0.025*"thing" + 0.015*"sit" + 0.014*"hate" + 0.013*"issue"
2022-05-17 09:04:05,329 : INFO : topic #13 (0.067): 0.049*"day" + 0.026*"thing" + 0.022*"start" + 0.016*"hour" + 0.016*"meltdown" + 0.015*"room" + 0.013*"use" + 0.013*"house" + 0.012*"leave" + 0.012*"morning"
2022-05-17 09:04:05,330 : INFO : topic diff=0.196066, rho=0.290020
2022-05-17 09:04:05,342 : INFO : PROGRESS: pass 8, at document #4000/5778
2022-05-

In [39]:
lda.print_topics();

2022-05-17 09:04:10,432 : INFO : topic #0 (0.067): 0.067*"kid" + 0.064*"mom" + 0.062*"parent" + 0.045*"family" + 0.041*"child" + 0.024*"sister" + 0.023*"dad" + 0.021*"mother" + 0.020*"cat" + 0.016*"school"
2022-05-17 09:04:10,433 : INFO : topic #1 (0.067): 0.041*"thing" + 0.039*"understand" + 0.025*"mean" + 0.023*"question" + 0.021*"word" + 0.020*"explain" + 0.018*"conversation" + 0.018*"person" + 0.013*"speak" + 0.013*"happen"
2022-05-17 09:04:10,434 : INFO : topic #2 (0.067): 0.043*"play" + 0.037*"watch" + 0.033*"music" + 0.030*"thing" + 0.027*"game" + 0.026*"noise" + 0.022*"character" + 0.021*"listen" + 0.020*"hear" + 0.020*"video"
2022-05-17 09:04:10,435 : INFO : topic #3 (0.067): 0.041*"friend" + 0.037*"year" + 0.028*"age" + 0.025*"dog" + 0.020*"thought" + 0.020*"adult" + 0.019*"thing" + 0.019*"walk" + 0.017*"realize" + 0.015*"use"
2022-05-17 09:04:10,436 : INFO : topic #4 (0.067): 0.033*"friend" + 0.032*"relationship" + 0.026*"thing" + 0.025*"love" + 0.024*"date" + 0.023*"husband

#### Decent results. I think NMF was still better.

# CorEx anchored topic modeling

LSA, NMF, and LDA all had a lot of overlap in the modeling of topics, with the repeating of common words (such as like, people, etc.) across topics. I used NMF and LDA groupings as ideas for my anchored topic modeling (CorEx). This produced the best results.

In [40]:
# CorEX anchored topic modeling for female autism forumns
vectorizer = CountVectorizer(stop_words='english')
doc_word = vectorizer.fit_transform(corpus_f_noun_verb_lem)
words = list(np.asarray(vectorizer.get_feature_names()))
topic_model = ct.Corex(n_hidden=24, words=words,
                       max_iter=5000, verbose=False, seed=1)

topic_model.fit(doc_word, words=words, docs=corpus_f_noun_verb_lem,
                anchors=[['diagnosis','diagnose'], 
                         ['therapist','therapy'],
                         ['conversation','speak','word'],
                         ['person','people'],
                         ['noise','music','sound'],
                         ['eye','contact'],
                         ['stim','hand'],
                         ['post','share','comment','read','write'],
                         ['anxiety','depression'],
                         ['meltdown','burnout'],
                         ['abuse','trauma'],
                         ['mask','masking'],
                         ['routine'],
                         ['home','house','live'],
                         ['friend','friendship'],
                         ['family','parent'], 
                         ['relationship','date','boyfriend','girlfriend'],                         
                         ['child','kid','daughter','son'],
                         ['school','teacher','class','college'],
                         ['work','job','interview'],
                         ['body','face','hair','wear'],
                         ['food','eat'],
                         ['night','sleep'],
                         ['cat','dog']], anchor_strength=10)

# Print all topics from the CorEx topic model
topics = topic_model.get_topics()
for n,topic in enumerate(topics):
    topic_words,_,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))

0: diagnose,diagnosis,autism,self,spectrum,suspect,assessment,symptom,research,psychologist
1: therapist,therapy,session,appointment,medication,insurance,patient,mindfulness,specialist,havetherapist
2: conversation,word,speak,phrase,topic,sentence,speech,script,repeat,mutism
3: people,person,interact,perceive,prefer,pleaser,participate,majority,impression,succeed
4: sound,noise,music,headphone,hear,listen,cancel,ear,song,voice
5: eye,contact,roll,close,stare,gaze,sun,nose,perfection,telehealth
6: hand,stim,flap,finger,stimme,toe,rock,stimming,stims,leg
7: post,read,share,write,comment,thank,book,reading,story,community
8: anxiety,depression,disorder,med,attack,illness,dysfunction,motivation,relieve,bout
9: meltdown,burnout,shutdown,recover,overstimulate,overstimulation,induce,verge,overwhelm,ward
10: trauma,abuse,make,think,say,know,way,try,lot,talk
11: mask,masking,unmask,personality,public,mandate,perform,persona,pandemic,version
12: routine,time,day,need,want,year,thing,start,week,h

In [41]:
# CorEX anchored topic modeling for general autism forumns
vectorizer = CountVectorizer(stop_words='english',min_df=0.01)
doc_word = vectorizer.fit_transform(corpus_g_noun_verb_lem)
words = list(np.asarray(vectorizer.get_feature_names()))
topic_model = ct.Corex(n_hidden=24, words=words,
                       max_iter=5000, verbose=False, seed=1)

topic_model.fit(doc_word, words=words, docs=corpus_g_noun_verb_lem,
                anchors=[['diagnosis','diagnose'], 
                         ['therapist','therapy'],
                         ['conversation','speak','word'],
                         ['person','people'],
                         ['noise','music','sound'],
                         ['eye','contact'],
                         ['stim','hand'],
                         ['post','share','comment','read','write'],
                         ['anxiety','depression'],
                         ['meltdown','burnout'],
                         ['abuse','trauma'],
                         ['mask','masking'],
                         ['time','routine'],
                         ['home','house','live'],
                         ['friend','friendship'],
                         ['family','parent'], 
                         ['relationship','date','boyfriend','girlfriend'],                         
                         ['child','kid','son'],
                         ['school','teacher','class','college'],
                         ['work','job','interview'],
                         ['body','face','hair','wear'],
                         ['food','eat'],
                         ['night','sleep'],
                         ['cat','dog']], anchor_strength=10)

# Print all topics from the CorEx topic model
topics = topic_model.get_topics()
for n,topic in enumerate(topics):
    topic_words,_,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))

0: diagnose,diagnosis,autism,symptom,doctor,test,adhd,suspect,spectrum,research
1: therapist,therapy,recommend,search
2: word,speak,conversation,speech,repeat,chat,piece
3: people,person,group,interact,empathy,human,making,label,neurotypical,public
4: sound,noise,music,ear,headphone,cancel,listen,voice,light,song
5: eye,contact,stare
6: hand,stim,stimme,touch
7: post,read,share,write,comment,book,thank,reading,sub,community
8: anxiety,depression,panic,attack
9: meltdown,burnout,calm,burn
10: trauma,abuse,make,know,thing,say,year,think,feel,way
11: mask,masking,pretend
12: time,routine,spend,period,waste,art,habit,tip,prefer,noticed
13: live,home,house,living,country,car,comfort,city,kill,switch
14: friend,friendship,hang,party,join
15: family,parent,member,dad,brother
16: relationship,date,girlfriend,boyfriend,girl,partner,maintain,sex
17: kid,child,son,adult,age,childhood,wife,baby
18: school,class,college,teacher,grade,student,study,degree,disability,university
19: work,job,interview

# Sentiment Analysis with VADER

In [42]:
analyzer = SentimentIntensityAnalyzer()
autism_fem['sentiment_score'] = autism_fem.full_clean.map(analyzer.polarity_scores).map(lambda x: x.get('compound'))
autism_fem.head()

,created_utc,title,selftext,full_text,spacy_doc,full_clean,nouns,adj,verb,propn,...,upvote_ratio,num_comments,ID,author,name,verb_lem,noun_lem,sentiment_score,full_text_nouns,category
0,2021-02-09 23:20:33,Origin Story Megathread,We get a lot of foot traffic from people who a...,Origin Story Megathread. We get a lot of foot ...,"(Origin, Story, Megathread, ., We, get, a, lot...",origin story megathread we get a lot of foot ...,lot foot traffic people process autism diagnos...,turbulent right medical individual new unusual...,get discovering investigating considering goin...,Origin Story Megathread,...,1.00,306,lggazi,idungonwent,t3_lggazi,get discover investigate consider go want know...,lot foot traffic people process autism diagnos...,0.9324,lot foot traffic process discovering investiga...,fem
1,2022-04-16 16:40:16,Study finds indicators that autistic people co...,**Lay abstract** \nSharing information with o...,Study finds indicators that autistic people co...,"(Study, finds, indicators, that, autistic, peo...",study finds indicators that autistic people co...,Study indicators people people communication d...,autistic fellow autistic allistic autistic abs...,finds commicate arise try communicate Sharing ...,Lay,...,0.98,26,u51o7g,Hoihe,t3_u51o7g,find commicate arise try communicate sharing r...,study indicator people people communication de...,0.9775,communication information ability communicatio...,fem
2,2022-04-16 17:38:28,"so, are your parents terrorizing you too?",lol i am trying to hold my tears back while wr...,"so, are your parents terrorizing you too?. lol...","(so, ,, are, your, parents, terrorizing, you, ...",so are your parents terrorizing you too lol...,parents tears x200B possibility diagnosis ther...,difficult dominant sure nicer neutral few rude...,terrorizing lol trying hold writing discussing...,,...,0.97,10,u52wf1,dykenergy,t3_u52wf1,terrorize lol try hold write discussing say di...,parent tear x200b possibility diagnosis therap...,-0.9584,lol i x200B possibility diagnosis therapist ma...,fem
3,2022-04-16 13:52:13,what are some of your social interactions that...,so obviously its no rare occasion for any of u...,what are some of your social interactions that...,"(what, are, some, of, your, social, interactio...",what are some of your social interactions that...,interactions occasion misunderstood NTs intera...,social wrong rare social own social social due...,gone get considering views tend blow think try...,faux pas H K K J. J K J J i J,...,0.98,38,u4y9ji,petpuppy,t3_u4y9ji,go get consider view tend blow think try find ...,interaction occasion misunderstood NTs interac...,-0.8480,occasion misunderstood faux pas proportion hum...,fem
4,2022-04-16 16:44:05,Human rights activist bullied for her autistic...,"I’ve never heard about this woman before, but ...",Human rights activist bullied for her autistic...,"(Human, rights, activist, bullied, for, her, a...",human rights activist bullied for her autistic...,rights activist traits woman People people thi...,Human autistic furious autistic weird childish...,bullied heard got read treated call try ’m hea...,Greta Thunberg 😡 https://hireup.com.au/news/gr...,...,0.93,2,u51r1d,Bitter-Statement6471,t3_u51r1d,bully hear got read treat call try ' m hear say ’,right activist trait woman People people thing...,-0.4019,activist woman sure https,fem


In [43]:
# highest score
highest_sentiment = autism_fem.iloc[autism_fem.sentiment_score.idxmax()]
print(highest_sentiment.full_text)

Its nice to have a little bit of friends again that actually understand me a bit and enjoy talking to me! -Also growing out of old friends-.. Sorry in advance if this post is a mess, i've been up since 3:30am and just thinking.  
So I'm not a complete shut in when it comes to socialization but I can supremely enjoy my own company if i have to.   
Other than sometimes not feeling like talking and shutting down a bit sometimes, I like talking to others a lot, and i would say im very good at it atleast 1 on 1.   


After losing a friend (they ended the friendship) and not understanding why even now, and after lockdowns and drifting from 2 others i found myself reevaluating my friendships in general, these are people I've known since childhood and highschool, but i realized that because i can empathize with almost anyone i can get along great with most people if i focus, but that i didn't really have anyone that enjoyed stuff i did like i can get into there stuff but i don't get that joy t

In [44]:
# lowest score
autism_fem.iloc[autism_fem.sentiment_score.idxmin()]
lowest_sentiment = autism_fem.iloc[autism_fem.sentiment_score.idxmin()]#['sentiment_score']
print(lowest_sentiment.full_text)

Response to "So you're going to be hospitalized involuntarily" from Someone Who Was.  

I have been involuntarily held at a juvenile psychiatric ward twice in my life. It was the same place both times, the first time for a week, and the second for a month. This post is a response to “So You’re Going to be Hospitalized Involuntarily”. Please don’t torment the writer of that one. I’m autistic, of course, so I can give my feedback on both my experience and her post. 

TRIGGER WARNING: Basically any graphic topic you can think of, including PTSD, forced medication, starvation, suicide, and bullying. Please excuse me, past OP of the post I’m responding to, if I’ve misinterpreted your words at all.

Addressing what they said: you’re not going to like it. **They are completely right.** Let me be clear that their post, to me and many others, reads as very distant, unempathetic, condescending, apathetic, and not focused on healing at all whatsoever for whoever must go into these places. They’re

In [45]:
# Average sentiment score of the female autism forums
autism_fem.sentiment_score.mean()

0.22287817583939035

In [46]:
analyzer = SentimentIntensityAnalyzer()
autism_gen['sentiment_score'] = autism_gen.full_text.map(analyzer.polarity_scores).map(lambda x: x.get('compound'))

# Average sentiment score of the general autism forums
autism_gen.sentiment_score.mean()

0.15154082550061274

### Difference in top words in general vs. female identified autism forums

#### Document Term Matrix

In [47]:
add_stop_words = ['autism', 'autistic', 'asd', 'like', 'just', 'really', 'don','t', 'i','m', 've', 'they','re', 
                  'd', 'isn', 'll', 'what','s', 'that', 'he', 'she','a', 'didn', 'wasn']

# Add new stop words
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)


In [48]:
# Create document term matrix from corpus
cv = CountVectorizer(stop_words=stop_words,min_df=0.01)
X = cv.fit_transform(corpus_f)
dtm_f = pd.DataFrame(X.toarray(), columns=cv.get_feature_names())

# Create dtm with totals in entire corpus
total_dtm_f = dtm_f.sum()  
total_dtm_f

ability        125
able           951
absolutely     278
abuse           86
abusive        102
              ... 
years         1276
yes            184
yesterday      194
young          188
younger        199
Length: 1159, dtype: int64

In [49]:
dtm_f

,ability,able,absolutely,abuse,abusive,accept,accepted,accepting,access,accommodations,...,written,wrong,wrote,yeah,year,years,yes,yesterday,young,younger
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5773,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5774,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5775,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5776,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
total_dtm_f.sort_values(ascending=False).head(20)

people     5715
feel       5363
know       4418
time       3548
things     2828
want       2763
think      2698
life       2030
work       2009
way        1996
lot        1983
make       1882
need       1615
does       1594
friends    1588
social     1576
help       1541
going      1517
got        1485
say        1483
dtype: int64

In [51]:
# Create document term matrix from corpus (includes title + paragraph text)
cv = CountVectorizer(stop_words=stop_words,min_df=0.01) 
X = cv.fit_transform(corpus_g)
dtm_g = pd.DataFrame(X.toarray(), columns=cv.get_feature_names())
total_dtm_g = dtm_g.sum()  
total_dtm_g.sort_values(ascending=False).head(20)

people     6742
feel       4595
know       4512
time       3296
want       2886
things     2858
think      2831
life       2142
make       1967
way        1955
does       1820
lot        1812
friends    1732
say        1709
help       1706
work       1622
social     1619
going      1554
person     1546
years      1492
dtype: int64

#### Interestingly, the forums for female-specific autism vs general population autism have a very strong overlap in their top terms. I checked for cross-posting and duplicate posts across the two datasets.
#### It will be necessary and interesting to see at what point the datasets differ.

In [52]:
# Find top words - nouns only

def collect_nouns(text):
    pos_list = pos_tag(word_tokenize(text))
    nouns = ''
    for item in pos_list:
        if item[1] == 'NN':
            nouns = nouns + item[0] + ' '
    return nouns 

autism_fem['full_text_nouns'] = autism_fem.full_text.map(collect_nouns)
autism_fem

,created_utc,title,selftext,full_text,spacy_doc,full_clean,nouns,adj,verb,propn,...,upvote_ratio,num_comments,ID,author,name,verb_lem,noun_lem,sentiment_score,full_text_nouns,category
0,2021-02-09 23:20:33,Origin Story Megathread,We get a lot of foot traffic from people who a...,Origin Story Megathread. We get a lot of foot ...,"(Origin, Story, Megathread, ., We, get, a, lot...",origin story megathread we get a lot of foot ...,lot foot traffic people process autism diagnos...,turbulent right medical individual new unusual...,get discovering investigating considering goin...,Origin Story Megathread,...,1.00,306,lggazi,idungonwent,t3_lggazi,get discover investigate consider go want know...,lot foot traffic people process autism diagnos...,0.9324,lot foot traffic process discovering investiga...,fem
1,2022-04-16 16:40:16,Study finds indicators that autistic people co...,**Lay abstract** \nSharing information with o...,Study finds indicators that autistic people co...,"(Study, finds, indicators, that, autistic, peo...",study finds indicators that autistic people co...,Study indicators people people communication d...,autistic fellow autistic allistic autistic abs...,finds commicate arise try communicate Sharing ...,Lay,...,0.98,26,u51o7g,Hoihe,t3_u51o7g,find commicate arise try communicate sharing r...,study indicator people people communication de...,0.9775,communication information ability communicatio...,fem
2,2022-04-16 17:38:28,"so, are your parents terrorizing you too?",lol i am trying to hold my tears back while wr...,"so, are your parents terrorizing you too?. lol...","(so, ,, are, your, parents, terrorizing, you, ...",so are your parents terrorizing you too lol...,parents tears x200B possibility diagnosis ther...,difficult dominant sure nicer neutral few rude...,terrorizing lol trying hold writing discussing...,,...,0.97,10,u52wf1,dykenergy,t3_u52wf1,terrorize lol try hold write discussing say di...,parent tear x200b possibility diagnosis therap...,-0.9584,lol i x200B possibility diagnosis therapist ma...,fem
3,2022-04-16 13:52:13,what are some of your social interactions that...,so obviously its no rare occasion for any of u...,what are some of your social interactions that...,"(what, are, some, of, your, social, interactio...",what are some of your social interactions that...,interactions occasion misunderstood NTs intera...,social wrong rare social own social social due...,gone get considering views tend blow think try...,faux pas H K K J. J K J J i J,...,0.98,38,u4y9ji,petpuppy,t3_u4y9ji,go get consider view tend blow think try find ...,interaction occasion misunderstood NTs interac...,-0.8480,occasion misunderstood faux pas proportion hum...,fem
4,2022-04-16 16:44:05,Human rights activist bullied for her autistic...,"I’ve never heard about this woman before, but ...",Human rights activist bullied for her autistic...,"(Human, rights, activist, bullied, for, her, a...",human rights activist bullied for her autistic...,rights activist traits woman People people thi...,Human autistic furious autistic weird childish...,bullied heard got read treated call try ’m hea...,Greta Thunberg 😡 https://hireup.com.au/news/gr...,...,0.93,2,u51r1d,Bitter-Statement6471,t3_u51r1d,bully hear got read treat call try ' m hear say ’,right activist trait woman People people thing...,-0.4019,activist woman sure https,fem
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5773,2022-05-11 15:54:15,Grimes,Anyone else love her? Not her music - but her ...,Grimes. Anyone else love her? Not her music - ...,"(Grimes, ., Anyone, else, love, her, ?, Not, h...",grimes anyone else love her not her music ...,music philosophy,,love watched,Grimes lex Awesomeness,...,0.40,34,und0ja,oksooosko,t3_und0ja,love watch,music philosophy,0.2732,music philosophy etc lex Awesomeness,fem
5774,2022-05-11 02:55:25,Constant sensory overload,Perhaps I didn’t notice or maybe I’ve just cha...,Constant sensory overload. Perhaps I didn’t n

In [53]:
autism_gen['full_text_nouns'] = autism_gen.full_text.map(collect_nouns)
autism_gen

,created_utc,title,selftext,full_text,spacy_doc,full_clean,nouns,adj,verb,propn,...,upvote_ratio,num_comments,ID,author,name,verb_lem,noun_lem,sentiment_score,full_text_nouns,category
0,2022-02-25 15:44:52,Any Ukrainian members here? Please drop a repl...,,Any Ukrainian members here? Please drop a repl...,"(Any, Ukrainian, members, here, ?, Please, dro...",any ukrainian members here please drop a repl...,members reply hub community,Ukrainian ok More,drop know have check,,...,0.99,115,t170dh,luis-mercado,t3_t170dh,drop know have check,member reply hub community,0.3400,reply hub community ..,gen
1,2022-04-16 07:00:09,Same Food Saturdays !,Most of us consistency love consistency. Shar...,Same Food Saturdays !. Most of us consistency...,"(Same, Food, Saturdays, !, ., Most, of, us, ,...",same food saturdays most of us consistency...,consistency love consistency go breakfast snac...,Same Most favorite good,Share have tried Share,Food Saturdays,...,0.95,15,u4s5m0,sybersonic,t3_u4s5m0,share have try Share,consistency love consistency go breakfast snac...,0.9335,consistency lunch dinner recipe something Share,gen
2,2022-04-16 14:50:09,irritable when not having enough alone time?,does anyone else have the issue where being ar...,irritable when not having enough alone time?. ...,"(irritable, when, not, having, enough, alone, ...",irritable when not having enough alone time ...,time issue people people time things walking m...,irritable enough alone long irritable snappy e...,having have love ends find do have do love tri...,,...,0.99,28,u4ze2j,bpdbryan,t3_u4ze2j,having have love end find do have do love trig...,time issue people people time thing walk music...,0.8805,time anyone issue time music point life,gen
3,2022-04-16 18:41:36,"My special interest is space, ask me anything!!!","I'm 13, and I've shown an interest in space si...","My special interest is space, ask me anything!...","(My, special, interest, is, space, ,, ask, me,...",my special interest is space ask me anything ...,interest space interest space astrophysicist,special,ask shown want become ask,,...,0.95,30,u548wh,Nova5000,t3_u548wh,ask show want become ask,interest space interest space astrophysicist,0.9210,interest space anything interest space astroph...,gen
4,2022-04-16 17:34:17,What’s something you would like (or would’ve l...,Trying to encourage my former teachers to make...,What’s something you would like (or would’ve l...,"(What, ’s, something, you, would, like, (, or,...",what s something you would like or would ve l...,’ve teachers autism teachers school people,former inclusive autistic,’s like liked know Trying encourage make,,...,1.00,38,u52t6c,rude_steppenwolf,t3_u52t6c,’s like liked know try encourage make,’ ve teacher autism teacher school people,0.8225,something ve autism school,gen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7336,2022-05-11 07:02:12,Anyone with psychosis here?,I was diagnosed with it at age 15.,Anyone with psychosis here?. I was diagnosed w...,"(Anyone, with, psychosis, here, ?, ., I, was, ...",anyone with psychosis here i was diagnosed w...,psychosis age,,diagnosed,,...,0.81,3,un3u34,Emergency_Sherbet831,t3_un3u34,diagnose,psychosis age,0.0000,Anyone psychosis age,gen
7337,2022-05-11 02:47:19,Rant from earlier,Copied and pasted the messages I just sent in ...,Rant from earlier. Copied and pasted the messa...,"(Rant, from, earlier, ., Copied, and, pasted, ...",rant from earlier copied and pasted the messa...,Rant messages discord dad story brother restau...,mad obvious angry mad mad open less mad mad up...,Copied pasted sent driving explaining involved...,,...,1.00,3,umzny0,Yeethanos,t3_umzny0,copy paste send driving explain involve like e...,rant message discord dad story brother restaur...,-0.9951,Rant discord dad story brother restaurant park...,gen
7338,2022-05-11 10:38:09,Does there is any gay guy here 18-36 years old?,I'm lonely and looking for someone to chat pre...,Does there is any gay 

In [54]:
corpus_noun_f = list(autism_fem.full_text_nouns)
corpus_noun_f


['lot foot traffic process discovering investigating autism diagnosis time discovery process track advice process experience story please story world superpower way autism spectrum diagnosis diagnosis process ',
 'communication information ability communication share information case information everyone everyone group group person group story share person person share everyone group story story stage share information information person interaction rapport person story finding share information experience rapport https //journals.sagepub.com/doi/full/10.1177/1362361320919286 ',
 'lol i x200B possibility diagnosis therapist mask obedience i i rocking face tone mum question i i tone problem i father i consent i therapy problem i presence i mind ',
 'occasion misunderstood faux pas proportion humor example i school i science grade year i class mind mind everything i observer participator troublemaker guy class incident mother choice time corner mother cancer guess i look face smirk face 

In [55]:
corpus_noun_f = list(autism_fem.full_text_nouns)

# Recreate document-term matrix

cv = CountVectorizer(stop_words=stop_words,min_df=0.01) #,min_df=0.01,ngram_range=(2, 3)
X = cv.fit_transform(corpus_noun_f)
dtm_n_f = pd.DataFrame(X.toarray(), columns=cv.get_feature_names())

# Create dtm with totals in entire corpus
dtm_total_n_f = dtm_n_f.sum()  

# Top words used in entire corpus
dtm_total_n_f.sort_values(ascending=False).head(20)

time          3455
life          1978
way           1972
lot           1950
day           1382
thing         1367
person        1354
diagnosis     1269
work          1183
job           1108
school         928
anxiety        836
family         814
friend         807
kind           752
today          743
experience     692
point          688
advice         667
home           667
dtype: int64

In [56]:
corpus_noun_g = list(autism_gen.full_text_nouns)

# Recreate document-term matrix

cv = CountVectorizer(stop_words=stop_words,min_df=0.01) #,min_df=0.01,ngram_range=(2, 3)
X = cv.fit_transform(corpus_noun_g)
dtm_n_g = pd.DataFrame(X.toarray(), columns=cv.get_feature_names())

# Create dtm with totals in entire corpus
dtm_total_n_g = dtm_n_g.sum()  

# Top words used in entire corpus
dtm_total_n_g.sort_values(ascending=False).head(20)

time         3234
life         2083
way          1936
lot          1779
person       1515
thing        1380
day          1314
school       1308
diagnosis    1060
job          1026
work          925
year          849
point         808
family        798
friend        792
anxiety       716
spectrum      715
https         704
kind          702
mom           658
dtype: int64

#### Again, there is a very strong overlap in top nouns in the forums for female-specific autism vs general population

In [57]:
# These are the top nouns in the female forum that are not in the general forum (top 200)
counter = 0
overlap_list = []
not_in_common = []
for term in dtm_total_n_f.sort_values(ascending=False).head(200).index:
    if term in dtm_total_n_g.sort_values(ascending=False).head(200).index:
        counter+=1
        overlap_list.append(term)
    else:
        not_in_common.append(term)
counter,not_in_common

(173,
 ['husband',
  'burnout',
  'thank',
  'mask',
  'trauma',
  'office',
  'masking',
  'meeting',
  'hell',
  'skin',
  'overload',
  'adhd',
  'cat',
  'routine',
  'career',
  'deal',
  'friendship',
  'period',
  'store',
  'daughter',
  'tone',
  'evaluation',
  'boss',
  'comfort',
  'weird',
  'manager',
  'title'])

In [58]:
# These are the top *words* in the female forum that are not in the general forum (top 200)
counter = 0
overlap_list = []
not_in_common = []
for term in total_dtm_f.sort_values(ascending=False).head(200).index:
    if term in total_dtm_g.sort_values(ascending=False).head(200).index:
        counter+=1
        overlap_list.append(term)
    else:
        not_in_common.append(term)
counter,not_in_common

(176,
 ['women',
  'share',
  'masking',
  'relate',
  'finally',
  'therapist',
  'super',
  'mask',
  'body',
  'questions',
  'food',
  'partner',
  'edit',
  'feelings',
  'nice',
  'health',
  'asking',
  'close',
  'house',
  'process',
  'overwhelmed',
  'tired',
  'face',
  'took'])

In [59]:
autism_fem.to_pickle('autism_fem.pkl')
autism_gen.to_pickle('autism_gen.pkl')

# Scattertext

In [60]:
autism_fem['category'] = 'fem'
autism_gen['category'] = 'gen'

In [61]:
merged_df = pd.concat([autism_fem, autism_gen], axis=0, join="inner")
merged_df.sample(5)

,created_utc,title,selftext,full_text,spacy_doc,full_clean,nouns,adj,verb,propn,...,upvote_ratio,num_comments,ID,author,name,verb_lem,noun_lem,sentiment_score,full_text_nouns,category
1888,2021-03-20 22:59:53,"""sToP mAkInG eXcUsEs"" Fuck you.",It would be nice if people would just accept t...,"""sToP mAkInG eXcUsEs"" Fuck you.. It would be n...","("", sToP, mAkInG, eXcUsEs, "", Fuck, you, .., I...",stop making excuses fuck you it would be n...,eXcUsEs Fuck people reasons trouble crap excus...,nice able other stubborn able fucking other,sToP accept give do having giving making condi...,,...,0.98,133,m9ifhg,mr_bigmouth_502,t3_m9ifhg,stop accept give do have give make condition o...,excuse Fuck people reason trouble crap excuse ...,0.2846,mAkInG eXcUsEs something trouble something cra...,gen
1597,2022-04-03 14:31:09,How to know if an Aspie guy likes me?,Hi! I would like to know what signs do you usu...,How to know if an Aspie guy likes me?. Hi! I w...,"(How, to, know, if, an, Aspie, guy, likes, me,...",how to know if an aspie guy likes me hi i w...,guy signs person person reason guy asperger at...,different general next,know likes like know try show know like know t...,Aspie,...,0.83,8,tvak9u,jeveuxavocado,t3_tvak9u,know like like know try show know like know te...,guy sign person person reason guy asperger att...,0.9746,guy Hi person person reason guy asperger attit...,gen
1583,2022-04-03 16:54:32,how to react if someone makes an unexpected gift?,I was at an art bazaar where I found a pair of...,how to react if someone makes an unexpected gi...,"(how, to, react, if, someone, makes, an, unexp...",how to react if someone makes an unexpected gi...,gift art bazaar pair earrings friend(I think f...,unexpected unique nice expensive anxious enoug...,react makes found saw struggling buying bought...,,...,0.82,13,tvdten,Cipra_lex_sed_lex,t3_tvdten,react make find see struggle buying buy give a...,gift art bazaar pair earring friend(I think fr...,0.9119,someone gift art bazaar pair friend friend mov...,gen
6864,2022-05-12 12:40:56,Hi gamers and bibliophiles 👋🏾,I just wanted to say hi to everyone in the com...,Hi gamers and bibliophiles 👋🏾. I just wanted t...,"(Hi, gamers, and, bibliophiles, 👋, 🏾., I, just...",hi gamers and bibliophiles :waving_hand_medium...,gamers bibliophiles community gaming shout fol...,special other favourite open visual spatial fa...,wanted say reading Thought like share playing ...,AFAB P&Cs Outer Worlds Wilds Borderlands Borde...,...,0.73,13,uo05k6,Ekun_Dayo,t3_uo05k6,want say read thought like share playing prefe...,gamer bibliophile community gaming shout folk ...,0.9918,hi everyone community shout folk share platfor...,gen
3980,2022-04-17 22:46:39,Just curious... how many of you are officially...,I know there's some people here that is in thi...,Just curious... how many of you are officially...,"(Just, curious, ..., how, many, of, you, are, ...",just curious how many of you are officially...,people sub advise questions option diagnosis k...,curious many last free like commercial single,diagnosed know 's looking asking added Feel co...,PS Poll](https://www.reddit.com,...,0.95,80,u5ya2i,CazaBobos,t3_u5ya2i,diagnose know be looking ask add Feel comment ...,people sub advise question option diagnosis ki...,0.8385,sub somebody advise option elaborate diagnosis...,gen


In [62]:
# Some quick text cleaning
merged_df.noun_lem =  [re.sub(r'[%s]' % re.escape(string.punctuation),' ', str(x)) for x in merged_df.noun_lem] # remove punctuation
merged_df.noun_lem =  [re.sub('\w*\d\w*',' ', str(x)) for x in merged_df.noun_lem] # remove words with a digit in them
merged_df.noun_lem =  [re.sub('x200B',' ', str(x)) for x in merged_df.noun_lem]
merged_df.noun_lem =  [re.sub('autism',' ', str(x)) for x in merged_df.noun_lem]
merged_df.noun_lem =  [re.sub('aspie',' ', str(x)) for x in merged_df.noun_lem]
merged_df.noun_lem =  [re.sub('https',' ', str(x)) for x in merged_df.noun_lem]
merged_df.noun_lem =  [re.sub('dae',' ', str(x)) for x in merged_df.noun_lem]
merged_df.noun_lem =  [re.sub('subreddit',' ', str(x)) for x in merged_df.noun_lem]
merged_df.noun_lem =  [re.sub('autistics',' ', str(x)) for x in merged_df.noun_lem]
merged_df.noun_lem =  [re.sub('aspergers',' ', str(x)) for x in merged_df.noun_lem]
merged_df.noun_lem =  [re.sub('aspergirl',' ', str(x)) for x in merged_df.noun_lem]
merged_df.noun_lem =  [re.sub('reddit',' ', str(x)) for x in merged_df.noun_lem]
merged_df.noun_lem =  [re.sub('autist',' ', str(x)) for x in merged_df.noun_lem]
merged_df.noun_lem =  [re.sub(' pm ',' ', str(x)) for x in merged_df.noun_lem]
merged_df.noun_lem =  [re.sub(' s ',' ', str(x)) for x in merged_df.noun_lem]
merged_df.noun_lem =  [re.sub('asperger',' ', str(x)) for x in merged_df.noun_lem]
merged_df.noun_lem =  [re.sub('woman',' ', str(x)) for x in merged_df.noun_lem]
merged_df.noun_lem =  [re.sub('[‘’“”…—]',' ', str(x)) for x in merged_df.noun_lem] # remove addt'l punctuation
merged_df.noun_lem =  [re.sub('\n',' ', str(x)) for x in merged_df.noun_lem] # remove newline chars
merged_df.noun_lem =  [re.sub('\xa0',' ', str(x)) for x in merged_df.noun_lem] # remove non-breaking space
merged_df.noun_lem =  [re.sub('\b[a-zA-Z]\b',' ', str(x)) for x in merged_df.noun_lem] # remove single letter words
merged_df.noun_lem =  [re.sub(' +', ' ',x) for x in merged_df.noun_lem] 


In [63]:
merged_df.head(50)

,created_utc,title,selftext,full_text,spacy_doc,full_clean,nouns,adj,verb,propn,...,upvote_ratio,num_comments,ID,author,name,verb_lem,noun_lem,sentiment_score,full_text_nouns,category
0,2021-02-09 23:20:33,Origin Story Megathread,We get a lot of foot traffic from people who a...,Origin Story Megathread. We get a lot of foot ...,"(Origin, Story, Megathread, ., We, get, a, lot...",origin story megathread we get a lot of foot ...,lot foot traffic people process autism diagnos...,turbulent right medical individual new unusual...,get discovering investigating considering goin...,Origin Story Megathread,...,1.00,306,lggazi,idungonwent,t3_lggazi,get discover investigate consider go want know...,lot foot traffic people process diagnosis time...,0.9324,lot foot traffic process discovering investiga...,fem
1,2022-04-16 16:40:16,Study finds indicators that autistic people co...,**Lay abstract** \nSharing information with o...,Study finds indicators that autistic people co...,"(Study, finds, indicators, that, autistic, peo...",study finds indicators that autistic people co...,Study indicators people people communication d...,autistic fellow autistic allistic autistic abs...,finds commicate arise try communicate Sharing ...,Lay,...,0.98,26,u51o7g,Hoihe,t3_u51o7g,find commicate arise try communicate sharing r...,study indicator people people communication de...,0.9775,communication information ability communicatio...,fem
2,2022-04-16 17:38:28,"so, are your parents terrorizing you too?",lol i am trying to hold my tears back while wr...,"so, are your parents terrorizing you too?. lol...","(so, ,, are, your, parents, terrorizing, you, ...",so are your parents terrorizing you too lol...,parents tears x200B possibility diagnosis ther...,difficult dominant sure nicer neutral few rude...,terrorizing lol trying hold writing discussing...,,...,0.97,10,u52wf1,dykenergy,t3_u52wf1,terrorize lol try hold write discussing say di...,parent tear possibility diagnosis therapist ma...,-0.9584,lol i x200B possibility diagnosis therapist ma...,fem
3,2022-04-16 13:52:13,what are some of your social interactions that...,so obviously its no rare occasion for any of u...,what are some of your social interactions that...,"(what, are, some, of, your, social, interactio...",what are some of your social interactions that...,interactions occasion misunderstood NTs intera...,social wrong rare social own social social due...,gone get considering views tend blow think try...,faux pas H K K J. J K J J i J,...,0.98,38,u4y9ji,petpuppy,t3_u4y9ji,go get consider view tend blow think try find ...,interaction occasion misunderstood NTs interac...,-0.8480,occasion misunderstood faux pas proportion hum...,fem
4,2022-04-16 16:44:05,Human rights activist bullied for her autistic...,"I’ve never heard about this woman before, but ...",Human rights activist bullied for her autistic...,"(Human, rights, activist, bullied, for, her, a...",human rights activist bullied for her autistic...,rights activist traits woman People people thi...,Human autistic furious autistic weird childish...,bullied heard got read treated call try ’m hea...,Greta Thunberg 😡 https://hireup.com.au/news/gr...,...,0.93,2,u51r1d,Bitter-Statement6471,t3_u51r1d,bully hear got read treat call try ' m hear say ’,right activist trait People people thing Peopl...,-0.4019,activist woman sure https,fem
5,2022-04-16 15:30:13,How do I tell my partner I don’t want to be hu...,I recently told my partner about my stims and ...,How do I tell my partner I don’t want to be hu...,"(How, do, I, tell, my, partner, I, do, n’t, wa...",how do i tell my partner i don t want to be hu...,partner partner stims relationship work stims ...,able fine happy first due crazy first scared able,tell want hugged stim told said want do said w...,,...,1.00,5,u507ci,Sweaty-Ad-3526,t3_u507ci,tell want hug stim tell say want do say want h...,partner partner stims relationship work stims ...,0.9808,partner t partner relationship anxiety enjoyme...,fem
6,20

In [64]:
corpus = st.CorpusFromPandas(
    merged_df,
    category_col = 'category',
    text_col = 'noun_lem',
    nlp=st.whitespace_nlp_with_sentences
).build()

In [65]:
html = st.produce_scattertext_explorer(
        corpus,
        category='fem',
        category_name='female',
        not_category_name='general',
        minimum_term_frequency=10,
        pmi_threshold_coefficient=5,
        width_in_pixels=1000,
        )


In [66]:
open('scattertext.html', 'wb').write(html.encode('utf-8'));

#### Doesn't look great... it might not be good for this dataset

In [67]:
autism_fem.to_pickle('autism_fem.pkl')
autism_gen.to_pickle('autism_gen.pkl')